# **Le mélanome de la peau**

## Introduction
   Un mélanome de la peau est une maladie des cellules de la peau appelées mélanocytes. Il se développe à partir d’une cellule initialement normale qui se transforme et se multiplie de façon anarchique pour former une lésion appelée tumeur maligne.

   Il existe quatre principaux types de mélanome de la peau : le mélanome superficiel extensif, le mélanome de Dubreuilh, le mélanome nodulaire et le mélanome acrolentigineux. Le traitement de ces différents types de mélanome repose essentiellement sur la chirurgie qui sera adaptée à la topographie (c’est-à-dire à l’endroit où est situé le mélanome) et à la profondeur de la lésion.

<figure>
<center>
<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/6/6c/Melanoma.jpg/260px-Melanoma.jpg' />
<figcaption>La mélanome de la peau</figcaption></center>
</figure>
</center>

## Formalisation des objectifs

## Acquisition des données

### Comprendre les données
Dans cette section, nous allons récupérer les données et étudier leurs distributions. Fait intéressant que les données contiennent à la fois des données tabulaires et des données d'image.

Nous continuerons en chargeant les métadonnées qui nous sont fournies. Les données de train ont 8 caractéristiques et 33126 observations,les données de test ont 5 caractéristiques et 10982 observations.

L'ensemble de données de train se compose de:

1. nom de l'image -> le nom de fichier de l'image spécifique pour la rame
2. patient_id -> identifie le patient unique
3. sexe -> sexe du patient
4. age_approx -> âge approximatif du patient au moment de la numérisation
5. anatom_site_general_challenge -> emplacement du site d'analyse
6. diagnostic -> informations sur le diagnostic
7. benign_malignant - indique le résultat du scan s'il est malin ou bénin
8. target -> même que ci-dessus mais mieux pour la modélisation car c'est binaire

### Analyser les données

In [ ]:
#Télécharger la bibliothèque nécessaire

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import pylab as pl
from IPython import display
import seaborn as sns
sns.set()
import re
import pydicom
import random
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import pandas as pd

#import pytorch_lightning as pl
from scipy.special import softmax
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score, auc

from skimage.io import imread
from PIL import Image
import plotly.express as px

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import os
import copy

from albumentations import Compose, RandomCrop, Normalize,HorizontalFlip, Resize
from albumentations import VerticalFlip, RGBShift, RandomBrightness
from albumentations.core.transforms_interface import ImageOnlyTransform
from albumentations.pytorch import ToTensor

from tqdm.notebook import tqdm

In [ ]:
os.listdir("../input/")

In [ ]:
basepath = '/kaggle/input/siim-isic-melanoma-classification/'
train_img_path = '/kaggle/input/siim-isic-melanoma-classification/jpeg/train/'
test_img_path = '/kaggle/input/siim-isic-melanoma-classification/jpeg/test/'
os.listdir(basepath)

In [ ]:
# Setting color palette.
color_black = ['#333300','#666600','#999900','#CCCC00','#FFFF00','#FFFF33','#FFFF66','#FFFF99','#FFFFCC']
color_black_2 = ['#FFD700','#BDB76B','#F0E68C','#EEE8AA','#FFEFD5','#FFDAB9','#330C73']

# Setting color palette.
orange_black = ['#fdc029', '#df861d', '#FF6347', '#aa3d01', '#a30e15', '#800000', '#171820']
color_black_3=['#318ce7','#ff66cc']
B = ['#C3073F','#1A1A1D']

In [ ]:
train_info = pd.read_csv(basepath + "train.csv")
train_info.head()

In [ ]:
test_info = pd.read_csv(basepath + "test.csv")
test_info.head()

Our test set misses three columns: diagnosis, benign_malignant & target.

In [ ]:
train_info.shape[0] / test_info.shape[0]

Three times more entries in train than in test.

In [ ]:
# Image names

# train: image_name count
train_info.image_name.value_counts().max()


In [ ]:
# test: image_name count
test_info.image_name.value_counts().max()

In [ ]:
# Patient id counts

# train: patient_id count
train_info.patient_id.value_counts().max()

In [ ]:
# test: patient_id count
test_info.patient_id.value_counts().max()

In [ ]:
patient_counts_train = train_info.patient_id.value_counts()
patient_counts_test = test_info.patient_id.value_counts()

fig, ax = plt.subplots(2,2,figsize=(20,12))

sns.distplot(patient_counts_train, ax=ax[0,0], color="orangered", kde=True);
ax[0,0].set_xlabel("Counts")
ax[0,0].set_ylabel("Frequency")
ax[0,0].set_title("Patient id value counts in train");

sns.distplot(patient_counts_test, ax=ax[0,1], color="lightseagreen", kde=True);
ax[0,1].set_xlabel("Counts")
ax[0,1].set_ylabel("Frequency")
ax[0,1].set_title("Patient id value counts in test");

sns.boxplot(patient_counts_train, ax=ax[1,0], color="orangered");
ax[1,0].set_xlim(0, 250)
sns.boxplot(patient_counts_test, ax=ax[1,1], color="lightseagreen");
ax[1,1].set_xlim(0, 250);

**Insights**
* For most of the patients we only have a few images ranging fom 1 to roughly 20.
* More than 45 images per patient is very seldom!
* Nonetheless we have patients with more than 100 images.
* There is one heavy outlier patient in the test set with close to 250 images.**

**Data Visualization Age Vs. Gender**

In [ ]:
fig, ax = plt.subplots(2,2,figsize=(20,15))

sns.boxplot(train_info.sex, train_info.age_approx, ax=ax[0,0], palette="Reds_r");
ax[0,0].set_title("Age per gender in train");

sns.boxplot(test_info.sex, test_info.age_approx, ax=ax[0,1], palette="Blues_r");
ax[0,1].set_title("Age per gender in test");

sns.countplot(train_info.age_approx, hue=train_info.sex, ax=ax[1,0], palette="Reds_r");
sns.countplot(test_info.age_approx, hue=test_info.sex, ax=ax[1,1], palette="Blues_r");

Il s'agit d'un boxplot qui décrit en jaune les males et en gris les femmes , comme il est bien clair la catégorie d'age entre 40ans et 60 ans des males ont plus de chances de ne pas avoir la maladie ,par contre les femmes non atteinte ont une categorie entre 35ans jusqu'a 50 ans. En contre partie les males atteint varie leur age entre 60 ans et 70 ans et les femmes atteintes entre 40 et 60 ans.

**A General Look With Sunburst Chart**

Sunburst chart is pretty cool looking fella I'd say. It also giving lots of basic information to us. Let's see...

* Only 2% of our targets are malignant
* On malignant images males are dominant with 62%
* Gender wise benign images are more balance 52-48% male female ratio
* Malignant image scan locations differs based on the patients gender:
* Meanwhile the torso is most common location in males it's almost half of the scans meanwhile in females it's 39%
* Lower extremity is more common with female scans than males 18% males vs 26% females
* Again upper extremity malignant scans is common with females than males (23- 17%)
* Benign image scan locations more similar between male and female patients.

In [ ]:
# Plotting interactive sunburst:


fig = px.sunburst(data_frame=train_info,
                  path=['benign_malignant', 'sex', 'anatom_site_general_challenge'],
                  color='sex',
                  color_discrete_sequence=orange_black,
                  maxdepth=-1,
                  title='Sunburst Chart Benign/Malignant > Sex > Location')

fig.update_traces(textinfo='label+percent parent')
fig.update_layout(margin=dict(t=0, l=0, r=0, b=0))
fig.show()

**Patient Information Management Individually**

In [ ]:
patient_gender_train = train_info.groupby("patient_id").sex.unique().apply(lambda l: l[0])
patient_gender_test = test_info.groupby("patient_id").sex.unique().apply(lambda l: l[0])

train_patients = pd.DataFrame(index=patient_gender_train.index.values, data=patient_gender_train.values, columns=["sex"])
test_patients = pd.DataFrame(index=patient_gender_test.index.values, data=patient_gender_test.values, columns=["sex"])

train_patients.loc[:, "num_images"] = train_info.groupby("patient_id").size()
test_patients.loc[:, "num_images"] = test_info.groupby("patient_id").size()

train_patients.loc[:, "min_age"] = train_info.groupby("patient_id").age_approx.min()
train_patients.loc[:, "max_age"] = train_info.groupby("patient_id").age_approx.max()
test_patients.loc[:, "min_age"] = test_info.groupby("patient_id").age_approx.min()
test_patients.loc[:, "max_age"] = test_info.groupby("patient_id").age_approx.max()

train_patients.loc[:, "age_span"] = train_patients["max_age"] - train_patients["min_age"]
test_patients.loc[:, "age_span"] = test_patients["max_age"] - test_patients["min_age"]

train_patients.loc[:, "benign_cases"] = train_info.groupby(["patient_id", "benign_malignant"]).size().loc[:, "benign"]
train_patients.loc[:, "malignant_cases"] = train_info.groupby(["patient_id", "benign_malignant"]).size().loc[:, "malignant"]
train_patients["min_age_malignant"] = train_info.groupby(["patient_id", "benign_malignant"]).age_approx.min().loc[:, "malignant"]
train_patients["max_age_malignant"] = train_info.groupby(["patient_id", "benign_malignant"]).age_approx.max().loc[:, "malignant"]

In [ ]:
train_patients.sort_values(by="malignant_cases", ascending=False).head()

**Images show**

In [ ]:
import cv2, pandas as pd, matplotlib.pyplot as plt
train = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
print('Examples WITH Melanoma')
imgs = train.loc[train.target==1].sample(10).image_name.values
plt.figure(figsize=(20,8))
for i,k in enumerate(imgs):
    img = cv2.imread(train_img_path+'/%s.jpg'%k)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    plt.subplot(2,5,i+1); plt.axis('off')
    plt.imshow(img)
plt.show()
print('Examples WITHOUT Melanoma')
imgs = train.loc[train.target==0].sample(10).image_name.values
plt.figure(figsize=(20,8))
for i,k in enumerate(imgs):
    img = cv2.imread(train_img_path+'/%s.jpg'%k)
    #img = cv2.imread('../input/siim-isic-melanoma-classification/jpeg/train/%s.jpg'%k)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    plt.subplot(2,5,i+1); plt.axis('off')
    plt.imshow(img)
plt.show()

### Préparation des données

In [ ]:
# Massing values

# Missing Train Values
missing_vals_train = train_info.isnull().sum() / train_info.shape[0]
missing_vals_train[missing_vals_train > 0].sort_values(ascending=False)


In [ ]:
# Missing Test Values
missing_vals_test = test_info.isnull().sum() / test_info.shape[0]
missing_vals_test[missing_vals_test > 0].sort_values(ascending=False)

1. The anatomy shows most missing values.



### Apprentissage (Modeling)

In [ ]:
sex_dummies = pd.get_dummies(train_info['sex'], prefix='sex')
train = pd.concat([train_info, sex_dummies], axis=1)

# getting dummy variables for gender on test set

sex_dummies = pd.get_dummies(test_info['sex'], prefix='sex')
test = pd.concat([test_info, sex_dummies], axis=1)

# dropping not useful columns

train.drop(['sex','image_name','patient_id','diagnosis','benign_malignant'], axis=1, inplace=True)
test.drop(['sex','image_name','patient_id'], axis=1, inplace=True)

In [ ]:
# getting dummy variables for location on train set

anatom_dummies = pd.get_dummies(train['anatom_site_general_challenge'], prefix='anatom')
train = pd.concat([train, anatom_dummies], axis=1)

# getting dummy variables for location on test set

anatom_dummies = pd.get_dummies(test['anatom_site_general_challenge'], prefix='anatom')
test = pd.concat([test, anatom_dummies], axis=1)

# dropping not useful columns

train.drop('anatom_site_general_challenge', axis=1, inplace=True)
test.drop('anatom_site_general_challenge', axis=1, inplace=True)


In [ ]:
import xgboost as xgb

from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score, cross_validate
from sklearn.metrics import roc_auc_score, roc_curve

In [ ]:
# dividing train set and labels for modelling

X = train.drop('target', axis=1)
y = train.target

In [ ]:
X.head()

**Setting Cross-Validation and Hold-out Set**
Cross validation might be enough but I wanted to test our model on data which it never seen before.


In [ ]:
# taking holdout set for validating with stratified y

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=42)

# 5 fold stratify for cv

cv = StratifiedKFold(5, shuffle=True, random_state=42)

In [ ]:
# setting model hyperparameters, didn't include fine tuning here because of timing reasons...

xg = xgb.XGBClassifier(
    n_estimators=750,
    min_child_weight=0.81,
    learning_rate=0.025,
    max_depth=2,
    subsample=0.80,
    colsample_bytree=0.42,
    gamma=0.10,
    random_state=42,
    n_jobs=-1,
)

In [ ]:
estimators = [xg]

In [ ]:
# cross validation scheme

def model_check(X_train, y_train, estimators, cv):
    model_table = pd.DataFrame()

    row_index = 0
    for est in estimators:

        MLA_name = est.__class__.__name__
        model_table.loc[row_index, 'Model Name'] = MLA_name

        cv_results = cross_validate(est,
                                    X_train,
                                    y_train,
                                    cv=cv,
                                    scoring='roc_auc',
                                    return_train_score=True,
                                    n_jobs=-1)

        model_table.loc[row_index,
                        'Train roc Mean'] = cv_results['train_score'].mean()
        model_table.loc[row_index,
                        'Test roc Mean'] = cv_results['test_score'].mean()
        model_table.loc[row_index, 'Test Std'] = cv_results['test_score'].std()
        model_table.loc[row_index, 'Time'] = cv_results['fit_time'].mean()

        row_index += 1

    model_table.sort_values(by=['Test roc Mean'],
                            ascending=False,
                            inplace=True)

    return model_table

#### **Model Results Based on Meta Features**
Results are encouraging! It seems little bit overfitting but we might fix that in future by fine tuning. Let's leave it like that for now...

In [ ]:
# display cv results

raw_models = model_check(X_train, y_train, estimators, cv)
print(raw_models)

In [ ]:
# fitting train data

xg.fit(X_train, y_train)

# predicting on holdout set
validation = xg.predict_proba(X_test)[:, 1]

# checking results on validation set
roc_auc_score(y_test, validation)

#### Creating Meta Submission


In [ ]:
# predicting on test set

predictions = xg.predict_proba(test)[:, 1]

In [ ]:
sample = pd.read_csv(os.path.join(base_path, 'sample_submission.csv'))

In [ ]:
# creating submission df

meta_df = pd.DataFrame(columns=['image_name', 'target'])

# assigning predictions on submission df

meta_df['image_name'] = sample['image_name']
meta_df['target'] = predictions

In [ ]:
# creating submission csv file

meta_df.to_csv('meta_with_img_data.csv', header=True, index=False)

###  CNN Neural Networks

In [ ]:
pip install -U segmentation-models

In [ ]:
import os
import random
import re
import math
import time

from tqdm import tqdm
from tqdm.keras import TqdmCallback


from pandas_summary import DataFrameSummary

import warnings

warnings.filterwarnings('ignore') # Disabling warnings for clearer outputs



seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)

In [ ]:
# Importing packages

import tensorflow as tf
import tensorflow.keras.backend as K
import efficientnet.tfkeras as efn
from kaggle_datasets import KaggleDatasets

seed_val = 42

tf.random.set_seed(seed_val)

In [ ]:
# Loading image storage buckets

# Data access
GCS_PATH = KaggleDatasets().get_gcs_path('siim-isic-melanoma-classification')
filenames_train = np.array(tf.io.gfile.glob(GCS_PATH + '/tfrecords/train*.tfrec')
filenames_test = np.array(tf.io.gfile.glob(GCS_PATH + '/tfrecords/test*.tfrec')

In [ ]:
# Setting TPU as main device for training, if you get warnings while working with tpu's ignore them.

DEVICE = 'TPU'
if DEVICE == 'TPU':
    print('connecting to TPU...')
    try:        
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        print('Could not connect to TPU')
        tpu = None

    if tpu:
        try:
            print('Initializing  TPU...')
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print('TPU initialized')
        except _:
            print('Failed to initialize TPU!')
    else:
        DEVICE = 'GPU'

if DEVICE != 'TPU':
    print('Using default strategy for CPU and single GPU')
    strategy = tf.distribute.get_strategy()

if DEVICE == 'GPU':
    print('Num GPUs Available: ',
          len(tf.config.experimental.list_physical_devices('GPU')))

print('REPLICAS: ', strategy.num_replicas_in_sync)
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
#you can edit these settings.

cfg = dict(
           batch_size=32,
           img_size=384,
    
           lr_start=0.000005,
           lr_max=0.00000125,
           lr_min=0.000001,
           lr_rampup=5,
           lr_sustain=0,
           lr_decay=0.8,
           epochs=12,
    
           transform_prob=1.0,
           rot=180.0,
           shr=2.0,
           hzoom=8.0,
           wzoom=8.0,
           hshift=8.0,
           wshift=8.0,
    
           optimizer='adam',
           label_smooth_fac=0.05,
           tta_steps=20
            
        )

In [ ]:
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift,
            width_shift):
    
    ''' Settings for image preparations '''

    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.

    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1], dtype='float32')
    zero = tf.constant([0], dtype='float32')
    rotation_matrix = tf.reshape(
        tf.concat([c1, s1, zero, -s1, c1, zero, zero, zero, one], axis=0),
        [3, 3])

    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape(
        tf.concat([one, s2, zero, zero, c2, zero, zero, zero, one], axis=0),
        [3, 3])

    # ZOOM MATRIX
    zoom_matrix = tf.reshape(
        tf.concat([
            one / height_zoom, zero, zero, zero, one / width_zoom, zero, zero,
            zero, one
        ],
                  axis=0), [3, 3])

    # SHIFT MATRIX
    shift_matrix = tf.reshape(
        tf.concat(
            [one, zero, height_shift, zero, one, width_shift, zero, zero, one],
            axis=0), [3, 3])

    return K.dot(K.dot(rotation_matrix, shear_matrix),
                 K.dot(zoom_matrix, shift_matrix))


def transform(image, cfg):
    
    ''' This function takes input images of [: , :, 3] sizes and returns them as randomly rotated, sheared, shifted and zoomed. '''

    DIM = cfg['img_size']
    XDIM = DIM % 2  # fix for size 331

    rot = cfg['rot'] * tf.random.normal([1], dtype='float32')
    shr = cfg['shr'] * tf.random.normal([1], dtype='float32')
    h_zoom = 1.0 + tf.random.normal([1], dtype='float32') / cfg['hzoom']
    w_zoom = 1.0 + tf.random.normal([1], dtype='float32') / cfg['wzoom']
    h_shift = cfg['hshift'] * tf.random.normal([1], dtype='float32')
    w_shift = cfg['wshift'] * tf.random.normal([1], dtype='float32')

    # GET TRANSFORMATION MATRIX
    m = get_mat(rot, shr, h_zoom, w_zoom, h_shift, w_shift)

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat(tf.range(DIM // 2, -DIM // 2, -1), DIM)
    y = tf.tile(tf.range(-DIM // 2, DIM // 2), [DIM])
    z = tf.ones([DIM * DIM], dtype='int32')
    idx = tf.stack([x, y, z])

    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m, tf.cast(idx, dtype='float32'))
    idx2 = K.cast(idx2, dtype='int32')
    idx2 = K.clip(idx2, -DIM // 2 + XDIM + 1, DIM // 2)

    # FIND ORIGIN PIXEL VALUES
    idx3 = tf.stack([DIM // 2 - idx2[0, ], DIM // 2 - 1 + idx2[1, ]])
    d = tf.gather_nd(image, tf.transpose(idx3))

    return tf.reshape(d, [DIM, DIM, 3])

def prepare_image(img, cfg=None, augment=True):
    
    ''' This function loads the image, resizes it, casts a tensor to a new type float32 in our case, transforms it using the function just above, then applies the augmentations.'''
    
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [cfg['img_size'], cfg['img_size']],
                          antialias=True)
    img = tf.cast(img, tf.float32) / 255.0

    if augment:
        if cfg['transform_prob'] > tf.random.uniform([1], minval=0, maxval=1):
            img = transform(img, cfg)

        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_saturation(img, 0.7, 1.3)
        img = tf.image.random_contrast(img, 0.8, 1.2)
        img = tf.image.random_brightness(img, 0.1)

    return img

In [ ]:
def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'image_name': tf.io.FixedLenFeature([], tf.string),
        'patient_id': tf.io.FixedLenFeature([], tf.int64),
        'sex': tf.io.FixedLenFeature([], tf.int64),
        'age_approx': tf.io.FixedLenFeature([], tf.int64),
        'anatom_site_general_challenge': tf.io.FixedLenFeature([], tf.int64),
        'diagnosis': tf.io.FixedLenFeature([], tf.int64),
        'target': tf.io.FixedLenFeature([], tf.int64),
        #'width': tf.io.FixedLenFeature([], tf.int64),
        #'height': tf.io.FixedLenFeature([], tf.int64)
    }

    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    return example['image'], example['target']


def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'image_name': tf.io.FixedLenFeature([], tf.string),
        'patient_id': tf.io.FixedLenFeature([], tf.int64),
        'sex': tf.io.FixedLenFeature([], tf.int64),
        'age_approx': tf.io.FixedLenFeature([], tf.int64),
        'anatom_site_general_challenge': tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    return example['image'], example['image_name']

def count_data_items(filenames):
    n = [
        int(re.compile(r'-([0-9]*)\.').search(filename).group(1))
        for filename in filenames
    ]
    return np.sum(n)

In [ ]:
def getTrainDataset(files, cfg, augment=True, shuffle=True):
    
    ''' This function reads the tfrecord train images, shuffles them, apply augmentations to them and prepares the data for training. '''
    
    ds = tf.data.TFRecordDataset(files, num_parallel_reads=AUTO)
    ds = ds.cache()

    if shuffle:
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        ds = ds.with_options(opt)

    ds = ds.map(read_labeled_tfrecord, num_parallel_calls=AUTO)
    ds = ds.repeat()
    if shuffle:
        ds = ds.shuffle(2048)
    ds = ds.map(lambda img, label:
                (prepare_image(img, augment=augment, cfg=cfg), label),
                num_parallel_calls=AUTO)
    ds = ds.batch(cfg['batch_size'] * strategy.num_replicas_in_sync)
    ds = ds.prefetch(AUTO)
    return ds

def getTestDataset(files, cfg, augment=False, repeat=False):
    
    ''' This function reads the tfrecord test images and prepares the data for predicting. '''
    
    ds = tf.data.TFRecordDataset(files, num_parallel_reads=AUTO)
    ds = ds.cache()
    if repeat:
        ds = ds.repeat()
    ds = ds.map(read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    ds = ds.map(lambda img, idnum:
                (prepare_image(img, augment=augment, cfg=cfg), idnum),
                num_parallel_calls=AUTO)
    ds = ds.batch(cfg['batch_size'] * strategy.num_replicas_in_sync)
    ds = ds.prefetch(AUTO)
    return ds

def get_model():
    
    ''' This function gets the layers inclunding efficientnet ones. '''
    
    model_input = tf.keras.Input(shape=(cfg['img_size'], cfg['img_size'], 3),
                                 name='img_input')

    dummy = tf.keras.layers.Lambda(lambda x: x)(model_input)

    outputs = []

    x = efn.EfficientNetB3(include_top=False,
                           weights='noisy-student',
                           input_shape=(cfg['img_size'], cfg['img_size'], 3),
                           pooling='avg')(dummy)
    x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    outputs.append(x)

    x = efn.EfficientNetB4(include_top=False,
                           weights='noisy-student',
                           input_shape=(cfg['img_size'], cfg['img_size'], 3),
                           pooling='avg')(dummy)
    x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    outputs.append(x)

    x = efn.EfficientNetB5(include_top=False,
                           weights='noisy-student',
                           input_shape=(cfg['img_size'], cfg['img_size'], 3),
                           pooling='avg')(dummy)
    x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    outputs.append(x)

    model = tf.keras.Model(model_input, outputs, name='aNetwork')
    model.summary()
    return model

In [ ]:
def compileNewModel(cfg):
    
    ''' Configuring the model with losses and metrics. '''    
    
    with strategy.scope():
        model = get_model()

    with strategy.scope():
        model.compile(optimizer=cfg['optimizer'],
                      loss=[
                          tf.keras.losses.BinaryCrossentropy(
                              label_smoothing=cfg['label_smooth_fac']),
                          tf.keras.losses.BinaryCrossentropy(
                              label_smoothing=cfg['label_smooth_fac']),
                          tf.keras.losses.BinaryCrossentropy(
                              label_smoothing=cfg['label_smooth_fac'])
                      ],
                      metrics=[tf.keras.metrics.AUC(name='auc')])
    return model


In [ ]:
def getLearnRateCallback(cfg):
    
    ''' Using callbacks for learning rate adjustments. '''
    
    lr_start = cfg['lr_start']
    lr_max = cfg['lr_max'] * strategy.num_replicas_in_sync * cfg['batch_size']
    lr_min = cfg['lr_min']
    lr_rampup = cfg['lr_rampup']
    lr_sustain = cfg['lr_sustain']
    lr_decay = cfg['lr_decay']

    def lrfn(epoch):
        if epoch < lr_rampup:
            lr = (lr_max - lr_start) / lr_rampup * epoch + lr_start
        elif epoch < lr_rampup + lr_sustain:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_rampup -
                                                lr_sustain) + lr_min
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=False)
    return lr_callback

def learnModel(model, ds_train, stepsTrain, cfg, ds_val=None, stepsVal=0):
    
    ''' Fitting things together for training '''
    
    callbacks = [getLearnRateCallback(cfg)]

    history = model.fit(ds_train,
                        validation_data=ds_val,
                        verbose=1,
                        steps_per_epoch=stepsTrain,
                        validation_steps=stepsVal,
                        epochs=cfg['epochs'],
                        callbacks=callbacks)

    return history

Here we train our model, takes a while but at the end we'll have strong model to make predictions!

In [ ]:
!pip install git+https://github.com/qubvel/efficientnet

In [ ]:
# getting train data

ds_train = getTrainDataset(
    filenames_train, cfg).map(lambda img, label: (img, (label, label, label)))
stepsTrain = count_data_items(filenames_train) / \
    (cfg['batch_size'] * strategy.num_replicas_in_sync)

# compiling and training model

model = compileNewModel(cfg)
learnModel(model, ds_train, stepsTrain, cfg)